In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-21 12:07:37 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-07-21 12:07:37 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-07-21 12:07:37 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

[2025-07-21 12:07:43,748] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-21 12:07:43 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpymd7z5cz/test.c -o /tmp/tmpymd7z5cz/test.o
2025-07-21 12:07:43 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpymd7z5cz/test.o -laio -o /tmp/tmpymd7z5cz/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-21 12:07:44 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmp5057atk5/test.c -o /tmp/tmp5057atk5/test.o
2025-07-21 12:07:44 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmp5057atk5/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmp5057atk5/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-21 12:07:45 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-21 12:07:45 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-21 12:07:45 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-21 12:07:45 wandb.docker.auth DEBUG    No config file found


In [4]:
# os.environ["BNB_CUDA_VERSION"] = "124"
# ! echo $BNB_CUDA_VERSION
# ! python -m bitsandbytes

In [5]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-21 12:07:50 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-21 12:07:50 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-07-21 12:07:50 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-21 12:07:50 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-07-21 12:08:36 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-21 12:08:37 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [6]:
from src.functional import free_gpu_cache

# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "64"

checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_params",
    f"{SYNTH_DATASET}",
    "_full__clamp=0.001",
    model_key.split("/")[-1],
)

version = "epoch_1"
# version = "final_model"

checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# loaded_deltas

free_gpu_cache()


d = loaded_deltas["model<>layers<>10<>mlp<>gate_proj"]
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [7]:
from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

#################################################
Trainable_CLS = TrainableLM_delta
# Trainable_CLS = TrainableLM_LoRA
#################################################

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-07-21 12:08:49 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-21 12:08:49 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-21 12:08:49 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-21 12:08:49 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-21 12:08:49 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-21 12:08:50 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-21 12:08:50 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
202

In [8]:
from src.selection.data  import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-21 12:08:54 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

In [114]:
from src.selection.data import SelectionSample, get_random_sample

sample = get_random_sample(
    people_by_category=people_by_category,
    mt=mt,
    n_distractors=5,
    get_alt_obj=True,
    # category="actor",
    # obj_idx=2,
    filter_by_lm_prediction=True,
)
print(sample)
print(sample.prompt, " >> ", f'"{sample.obj}"')
sample.prediction

Mike Pence -> Josh Hawley (2): ['Anthony Davis', 'Jane Mayer', 'Josh Hawley', 'Chuck Todd', 'Paulo Dybala', 'Marc Jacobs']
Which person from the following list has the profession in common with Mike Pence?
Options: Anthony Davis, Jane Mayer, Josh Hawley, Chuck Todd, Paulo Dybala, Marc Jacobs.
Ans:  >>  "Josh Hawley"


[PredictedToken(token=' Josh', prob=0.70703125, logit=19.25, token_id=18747, metadata=None),
 PredictedToken(token=' Mike', prob=0.05810546875, logit=16.75, token_id=11519, metadata=None),
 PredictedToken(token=' The', prob=0.045166015625, logit=16.5, token_id=578, metadata=None),
 PredictedToken(token=' Chuck', prob=0.039794921875, logit=16.375, token_id=34349, metadata=None),
 PredictedToken(token=' None', prob=0.039794921875, logit=16.375, token_id=2290, metadata=None)]

## ATTN pattrn for single example

In [115]:
from src.tokens import prepare_input
from src.attention import get_attention_matrices
from src.functional import interpret_logits

tokenized = prepare_input(
    tokenizer=mt,
    prompts=sample.prompt,
    return_offsets_mapping=True
)

attn_matrices = get_attention_matrices(
    input = tokenized,
    mt = mt,
    value_weighted=True,
    # patches = []
)

interpret_logits(tokenizer=mt, logits = attn_matrices.logits)

odict_keys(['last_hidden_state', 'past_key_values', 'attentions'])
torch.Size([128256])
2025-07-21 15:13:20 src.attention DEBUG    attentions.shape=torch.Size([80, 64, 41, 41]) | values.shape=torch.Size([80, 64, 41, 128])


[PredictedToken(token=' Josh', prob=0.70703125, logit=19.25, token_id=18747, metadata=None),
 PredictedToken(token=' Mike', prob=0.05810546875, logit=16.75, token_id=11519, metadata=None),
 PredictedToken(token=' The', prob=0.045166015625, logit=16.5, token_id=578, metadata=None),
 PredictedToken(token=' Chuck', prob=0.039794921875, logit=16.375, token_id=34349, metadata=None),
 PredictedToken(token=' None', prob=0.039794921875, logit=16.375, token_id=2290, metadata=None)]

In [116]:
attn_matrices.attention_matrices.shape

(80, 64, 41, 41)

In [117]:
from src.attention import visualize_average_attn_matrix

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt = sample.prompt,
    tokenized=tokenized,
    layer_window=range(0, mt.n_layer, 1)
)

layer=0


--------------------------------------------------------------------------------
layer=1


--------------------------------------------------------------------------------
layer=2


--------------------------------------------------------------------------------
layer=3


--------------------------------------------------------------------------------
layer=4


...

--------------------------------------------------------------------------------
layer=76


--------------------------------------------------------------------------------
layer=77


--------------------------------------------------------------------------------
layer=78


--------------------------------------------------------------------------------
layer=79


--------------------------------------------------------------------------------


In [101]:
tokenized = prepare_input(
    tokenizer=mt,
    prompts=sample.prompt,
    return_offsets_mapping=True,
)
offset_mapping = tokenized.pop("offset_mapping")[0]


In [102]:
from src.tokens import find_token_range

subj_range = find_token_range(
    string=sample.prompt,
    substring=sample.subj,
    offset_mapping=offset_mapping,
    tokenizer=mt,
    occurrence=-1,
)

print(
    f'{subj_range=} | "{mt.tokenizer.decode(tokenized.input_ids[0][range(*subj_range)])}"'
)

option_ranges = []

for option in sample.options:
    option_range = find_token_range(
        string=sample.prompt,
        substring=option,
        offset_mapping=offset_mapping,
        tokenizer=mt,
        occurrence=-1,
    )
    print(
        f'{option_range=} | "{mt.tokenizer.decode(tokenized.input_ids[0][range(*option_range)])}"'
    )
    option_ranges.append(option_range)

subj_range=(13, 16) | " Johnny Depp"
option_range=(19, 21) | " Marc Jacobs"
option_range=(22, 24) | " Jennifer Lawrence"
option_range=(25, 33) | " Giannis Antetokounmpo"
option_range=(34, 36) | " Beyoncé"
option_range=(37, 39) | " Stephen King"
option_range=(40, 45) | " Novak Djokovic"


In [103]:
from src.functional import patchscope, get_hs
from itertools import product

pred_range = (-2, 0)
token_ranges = [pred_range, subj_range] + option_ranges
token_indices = []
for rng in token_ranges:
    token_indices.extend(list(range(rng[0], rng[1])))

layers = mt.layer_names
locations = list(product(layers, token_indices))

hs = get_hs(mt=mt, input=tokenized, locations=locations, patches=[], return_dict=True)


########################################################################
context_prof = """Name of a person -> their profession
Albert Einstein -> physicist
Michael Jordan -> basketball player
Jack Ma -> entrepreneur
placeholder ->"""

patch_layers = [5]
patch_layers = [mt.layer_name_format.format(idx) for idx in patch_layers]
########################################################################

context = context_prof
context_tok = prepare_input(
    prompts=context_prof, tokenizer=mt, return_offsets_mapping=True
)
cxt_offset_mapping = context_tok.pop("offset_mapping")[0]

h_idx = (
    find_token_range(
        string=context,
        substring="placeholder",
        offset_mapping=cxt_offset_mapping,
        tokenizer=mt.tokenizer,
        occurrence=-1,
    )[1]
    - 1
)

for layer in mt.layer_names[::3]:
    print(f"\n{layer=}")
    for cur_rng in token_ranges:
        print(
            f'\n"{mt.tokenizer.decode(tokenized.input_ids[0][range(*cur_rng)])}" | {cur_rng=}'
        )
        for token_idx in range(cur_rng[0], cur_rng[1]):
            token = mt.tokenizer.decode(tokenized.input_ids[0][token_idx])
            ll_pred = patchscope(
                mt=mt,
                h=hs[(layer, token_idx)],
                context=context,
                placeholder="placeholder",
                context_tokenized=context_tok,
                placeholder_idx=h_idx,
                patch_layers=patch_layers,
                k=15,
            )
            ll_fmt = [
                f'"{pred.token}"[p={pred.prob:.2f}, l={pred.logit:.2f}]'
                for pred in ll_pred
            ]
            print(f'{token_idx=} ["{token}"] => {ll_fmt}')

    print("=" * 150)


layer='model.layers.0'

"Ans:" | cur_rng=(-2, 0)
token_idx=-2 ["Ans"] => ['"?\n"[p=0.05, l=11.25]', '" programmer"[p=0.04, l=11.06]', '"??"[p=0.03, l=10.88]', '" software"[p=0.03, l=10.81]', '"???"[p=0.03, l=10.75]', '" unknown"[p=0.03, l=10.69]', '" student"[p=0.02, l=10.50]', '" entrepreneur"[p=0.02, l=10.44]', '" singer"[p=0.02, l=10.44]', '" artist"[p=0.02, l=10.38]', '"?\n\n"[p=0.02, l=10.31]', '"?"[p=0.02, l=10.31]', '" ("[p=0.01, l=10.06]', '" actor"[p=0.01, l=10.06]', '" computer"[p=0.01, l=10.00]']
token_idx=-1 [":"] => ['"?\n"[p=0.20, l=13.62]', '"?\n\n"[p=0.15, l=13.31]', '"??"[p=0.10, l=12.88]', '"?"[p=0.09, l=12.81]', '"???"[p=0.07, l=12.50]', '" \n"[p=0.02, l=11.50]', '" lawyer"[p=0.02, l=11.44]', '" unknown"[p=0.01, l=11.00]', '" artist"[p=0.01, l=10.69]', '" politician"[p=0.01, l=10.56]', '" software"[p=0.01, l=10.50]', '" programmer"[p=0.01, l=10.38]', '" \n\n"[p=0.01, l=10.31]', '" ("[p=0.01, l=10.25]', '" "[p=0.01, l=10.19]']

" Johnny Depp" | cur_rng=(13, 16)
token